In [39]:
import getpass
import json
import numpy as np
import pandas as pd
import pickle as pkl
import tiktoken
import tqdm

from collections import defaultdict
from data_processing import pers_labels
from openai import OpenAI
from sklearn.metrics import f1_score, precision_score, recall_score

In [7]:
client = OpenAI(api_key=getpass.getpass("please enter your openai api key"))

In [189]:
PWD = os.environ["WORKSPACE_PATH"]

data_type = "big 5_4_all_lbls"

label_mode = pers_labels.MBPT if pers_labels.MBPT.lower() in data_type else pers_labels.BIG_5

index = [idx for idx in range(5) if str(idx) in data_type][0]

pers_defs = {
    pers_labels.MBPT: {
        0: (("I", "introverted"), ("E", "extroverted")),
        1: (("S", "sensing"), ("N", "intuitive")),
        2: (("F", "feeling"), ("T", "thinking")),
        3: (("J", "judging"), ("P", "perceiving")),
    },
    pers_labels.BIG_5: {
        0: (("S", "social"), ("R", "reserved")),
        1: (("L", "limbic"), ("C", "calm")),
        2: (("O", "organized"), ("U", "unstructured")),
        3: (("A", "agreeable"), ("E", "egocentric")),
        4: (("N", "non-curious"), ("I", "inquisitive")),
    }
}

((label1, label1_def), (label2, label2_def)) = pers_defs[label_mode][index]

## loading data

In [190]:
with open(f"{PWD}/data/cornell_movies/speakers.json", "r+") as fp:
    fp_parsed = json.load(fp)
    chars_meta = {}
    chars_meta_rows = []
    for char in fp_parsed:
        meta = fp_parsed[char]["meta"]
        meta["character_name"] = meta["character_name"].lower()
        meta["char_id"] = char
        chars_meta[char] = meta
        chars_meta_rows.append(meta)

In [191]:
dataset = pd.read_json(f"{PWD}/data/datasets/{data_type}.jsonl", lines=True)

## query gpt

In [192]:
def classify_text(text, char, num_votes):

    _, scene = text.split("\n", 1)

    # To get the tokeniser corresponding to a specific model in the OpenAI API:
    enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

    prompt = f"""
Read the scenes below and then categorize {char}'s personality as {label1} for {label1_def} or "{label2}" for {label2_def}, according to the {label_mode.lower()} personality typology. Response with only one word.

scenes:
    {scene}
    """

    prompt = enc.decode(enc.encode(prompt)[:4000])

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # GPT-3.5 Turbo engine
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1,
        n=num_votes,
    )

    # Extract the first choice (response)
    predicted_label = [choice.message.content for choice in response.choices]
    return prompt, predicted_label

In [193]:
chars = [char_id for char_id in chars_meta]
preds = defaultdict(list)
labels = defaultdict(list)

In [194]:
for char_id in tqdm.tqdm(dataset.char_id.unique()):
    if char_id in preds:
        continue
    char_name = chars_meta[char_id]["character_name"]

    char_rows = dataset[dataset.char_id == char_id]

    char_labels = char_rows.label.tolist()

    scene = dataset.text[0]

    prompt, response = classify_text(scene, char_name, min(100, len(char_labels)))

    votes = [r.upper() for r in response if r.upper() in [label1, label2]]

    # print(prompt)
    # print()

    # print("Correct:", row.label)
    # print("Predicted:", response)
    preds[char_id] = votes
    labels[char_id] = char_labels

100%|██████████| 866/866 [08:45<00:00,  1.65it/s]


In [195]:
with open(f"{PWD}/data/gpt_preds/{data_type}.json", "w+") as fp:
    json.dump(preds, fp)

## evaluation

### overall metrics

In [196]:
preds_len = len(preds)
# preds_len = 1897

with open(f"{PWD}/data/gpt_preds/{data_type}.json", "r+") as fp:
    old_preds = json.load(fp)

In [197]:
labels = {}
preds = {}

for char_id in chars:
    if len(old_preds.get(char_id, [])) > 0:
        labels[char_id] = dataset[dataset.char_id == char_id].label.tolist()
        preds[char_id] = old_preds[char_id]

In [199]:
with open(f"../data/label_encoders/{data_type.replace('all_lbls', 'top_lbl')}.pkl", "rb+") as fp:
    label_enc = pkl.load(fp)

pred_array = [label_enc.transform([s.upper() for s in preds[char_id]]) for char_id in preds]
label_array = [label_enc.transform(labels[char_id]) for char_id in labels]

FileNotFoundError: [Errno 2] No such file or directory: '../data/label_encoders/big 5_4_all_lbls.pkl'

In [ ]:
print(len(pred_array), len(label_array))

862 862


In [ ]:
pred_means = [np.mean(char_lbls) for char_lbls in pred_array]
label_means = [np.mean(char_lbls) for char_lbls in label_array]

In [ ]:
pred_sample_mean_mean = np.mean(pred_means)
label_sample_mean_mean = np.mean(label_means)

print(pred_sample_mean_mean)
print(label_sample_mean_mean)

0.8121660231063195
0.4969408783160789


In [ ]:
pred_sample_mean_std = np.std(pred_means)
label_sample_mean_std = np.std(label_means)

print(pred_sample_mean_std)
print(label_sample_mean_std)

0.3002969691467325
0.46735324985832877


In [ ]:
# pred_sample_mean_std = np.std(pred_means)
# label_sample_mean_std = np.std(label_means)

# print(pred_sample_mean_std)
# print(label_sample_mean_std)

In [ ]:
# pred_sample_std_mean = np.std(pred_means)
# label_sample_std_mean = np.std(label_means)

# print(pred_sample_mean_std)
# print(label_sample_mean_std)